In [1]:
#Import Packages
import pandas as pd
import numpy as np
from skimage import io
import validators
from numpy.core.numeric import False_, True_
import dash
from dash import dcc
from dash import html
from dash import dash_table as dt
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
from datetime import date
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn import cluster
plt.style.use('default')

# Import and Clean Data

### Import and do initial cleaning and transformations

In [2]:
#Read in movie data
df = pd.read_excel(r'C:\Users\nickp\Downloads\movie_data.xlsx')

#read in country code mapping dataframe
country_code_df = pd.read_excel(r'C:\Users\nickp\Desktop\usc\fall 2021 semester\statistical computing and data visualization\movie_data_nick.xlsx', 
                                sheet_name = 'country codes')
df.head()

,title,original_title,year,date_published,genre,duration,country,Released_in_USA?,language,director,...,avg_vote,votes,budget_mixed_currencies,currency_code,budget_converted_to_usd,usa_gross_income,worldwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,The Devil's Doorway,The Devil's Doorway,2018,2019-02-28 00:00:00,Horror,76,"Ireland, UK",0,English,Aislinn Clarke,...,5.2,2571,1,USD,1.0,NaN,516660,48.0,48.0,45.0
1,Firincinin Karisi,Firincinin Karisi,2019,2019-09-27 00:00:00,Comedy,91,Turkey,0,Turkish,Murat Onbul,...,2.9,728,2,USD,2.0,NaN,471182,NaN,NaN,1.0
2,Fetih 1453,Fetih 1453,2012,2012-02-16 00:00:00,"Action, Drama, History",162,Turkey,0,Turkish,Faruk Aksoy,...,6.7,56173,18,USD,18.0,35730.0,35797045,NaN,154.0,21.0
3,The Exiles,The Exiles,1961,1961-07-13 00:00:00,Drama,72,USA,1,English,Kent Mackenzie,...,6.7,988,539,USD,539.0,30945.0,30945,NaN,13.0,53.0
4,The Last Broadcast,The Last Broadcast,1998,1998-10-23 00:00:00,"Horror, Mystery",86,USA,1,English,"Stefan Avalos, Lance Weiler",...,5.3,4744,900,USD,900.0,12097.0,12097,NaN,171.0,46.0


In [3]:
#Filter to keep rows only over 100k for gross income and budget 
df = df.loc[(df['budget_converted_to_usd']>=10000) & (df['worldwide_gross_income']>=10000)]

#Calculate the ROI
df['ROI'] = (df['worldwide_gross_income']-df['budget_converted_to_usd'])/df['budget_converted_to_usd']
df['ROI'] = round(df['ROI'],2)

In [4]:
#Create new column classifying genres as either multiple or the one they were, as commas indicate multiple genre
df['new genre'] = np.where(df['genre'].str.contains(','), 'Multiple', df['genre'])

In [5]:
#Map release location numbers to categories for mike's charts
df['Release Location'] = np.where(df['Released_in_USA?'] == 1, 'USA', 'Not USA')

In [6]:
#create decades column
df['Decade'] = df['year'].astype(str).str[:3] + '0\'s'

In [7]:
#create a cleaned country column for map on summary page
df['country updated'] = df['country'].str.split(',').str[0]

In [8]:
#Send country code dataframe to dictionary to use to map to df
country_code_dict = country_code_df.set_index('country')['country_code'].to_dict()

#Map country code column to df
df['country_code'] = df['country updated'].map(country_code_dict)

### Column Renaming After Cleaning

In [ ]:
df.rename(columns={'budget_converted_to_usd':'Budget in USD',
                        'worldwide_gross_income':'Worldwide Gross Income',
                        'usa_gross_income':'US Gross Income',
                        'new genre':'Genre with Multiple Category'}, inplace=True)

# Dash App Run

#### Define color scheme

In [ ]:
#Set two primary colors
color1 = 'red'
color2 = 'blue'

#Headers of each tab color
header_color = 'rgb(135,206,250)'

#Secondary section headers 
section_header_color = 'rgb(240,128,128)'

#Headers for any tables
table_header_color = 'rgb(173,216,230)'


#background color for what is around the image
title_page_background_color = 'rgb(238, 64, 53)'

In [ ]:
#Set a default image
default_title_image = 'https://upload.wikimedia.org/wikipedia/commons/thumb/5/5a/Hollywood_Sign_%28Zuschnitt%29.jpg/1200px-Hollywood_Sign_%28Zuschnitt%29.jpg'

#### Lists used in the app

In [ ]:
#Define what is classified as numeric columns
numeric_types = ['float64', 'int64']


#Categorical columns for first tab
categorical_columns = df.select_dtypes(exclude=numeric_types)


#List for columns to switch for y_axis in scatterplot with release location filter
numeric_columns = df.select_dtypes(include=numeric_types)

#### Run app

In [ ]:
##DESCRIPTION
#This app is broken up by tabs, and the first part of the app is setting up the layout
#The second part is in the order of tabs, assigning the necessary callbacks and functions to execute them
#Further detail on each tab, callback and function is provided in the code

In [ ]:
#Initialize the app
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

#Break into the desired number of tabs - tabs are static, only call backs are on different graphs
app.layout = html.Div([
    dcc.Tabs([
        #This first tab is a title tab
        dcc.Tab(label='Title', children=[
            dbc.Row([
            html.H1('Movie Analytics Dashboard', 
                    style={'textAlign': 'center', 
                           "font-size": 36,
                          'backgroundColor':header_color}
                   )
            ]),
            dbc.Row([
            html.H3('Key Drivers of ROI in the Movie Industry', 
                    style={'textAlign': 'center', 
                           "font-size": 24,
                           'backgroundColor':section_header_color})
            ]),
            dbc.Row([
                dcc.Graph(id='title-image',
                     style={'backgroundColor':title_page_background_color})
            ]),
            dbc.Row([
                dbc.Col([
                    dcc.Input(id="title-selector",
                         type="url",
                         value=default_title_image)
                    ], width=10, style= {'display': 'block'}),
                dbc.Col([
                    dcc.Dropdown(
                        id = 'dropdown-to-show_or_hide-element',
                        options=[
                            {'label': 'Show url', 'value': 'on'},
                            {'label': 'Hide url', 'value': 'off'}
                        ],
                        value = 'off'
                    )
                ], width=2),
            ]),
        ]),
        
        #Second tab is taking a look at categorical columns and how ROI differs.
        #Also gives a stacked bar chart showing the distribution of each category
        dcc.Tab(label='Categorical Analysis', children=[
            html.H2('Qualitative Factors and ROI', 
                    style={'textAlign': 'center',
                          'backgroundColor':header_color}),
            dbc.Row([
                dbc.Col([
                    html.Label('Please select max ROI to include')
                ]),
                dbc.Col([
                    html.Label('Please category for analysis')
                ])
            ]),
            dbc.Row([
                dbc.Col([
                    dcc.Input(id="filter-roi-categorical",
                        type='number', 
                        min=df['ROI'].min(),
                        max=df['ROI'].max(), 
                        value=10)
                        ], width=6),
                dbc.Col([
                    dcc.Dropdown(
                        id='categorical-variable',
                        options=[{'label': i, 'value': i} for i in categorical_columns],
                        value='Genre with Multiple Category')
                ])
            ]),
            html.H3('ROI Distribution by Category', 
                    style={'textAlign': 'center', 
                           "font-size": 24,
                           'backgroundColor':section_header_color}),
            dcc.Graph(id = 'categorical-boxplot'),
            html.H3('Distribution', 
                    style={'textAlign': 'center', 
                           "font-size": 24,
                           'backgroundColor':section_header_color}),
            dbc.Row([
                dbc.Col([
                    dcc.Graph(id = 'categorical-barplot')
                ]),
                dbc.Col([
                    dcc.Graph(id = 'categorical-piechart')
                ]),
            ])            
        ]),
        
        
        #Third tab is a focus on numerical columns
        #first chart is a histogram of numeric columns, and second chart is a scatterplot
        dcc.Tab(label='Quantitative Analysis', children=[
            html.H2('Quantitative Data and ROI', 
                    style={'textAlign': 'center',
                          'backgroundColor':header_color}),
            dbc.Row([
                dbc.Col([html.Label('Please select the desired numeric column:')
                        ]),
                dbc.Col([html.Label('Please select the maximum roi to include:')
                        ])
            ]),
            dbc.Row([
                dbc.Col([
                    dcc.Dropdown(
                        id='x-axis-numeric',
                        options=[{'label': i, 'value': i} for i in numeric_columns if i != 'ROI'],
                        value='Worldwide Gross Income'
                        ),
                        ], width=6),
                dbc.Col([
                    dcc.Input(id="max-roi-numerics",
                        type='number', 
                        min=df['ROI'].min(),
                        max=df['ROI'].max(), 
                        value=5)
                        ], width=6)
            ]),
            dbc.Row([
                dbc.Col([
                    dcc.Graph(id = 'numeric-histogram')
                ]),
                dbc.Col([
                    dcc.Graph(id = 'relationship-scatterplot-graph')
                ])
            ])
        ]),
        
        #Fourth tab is a k means clustering analysis of budget and roi
        #Can select number of clusters and roi max
        #first chart is scatter plot of clusters by budget and roi, second is elbow method to evaluate optimal clusters
        dcc.Tab(label='Cluster Analysis', children = [
             dbc.Card(
                dbc.CardBody([
                    dbc.Row([
                        dbc.Col([
                            html.H2('K Means Clustering of Movies by Budget and ROI', 
                                    style={'textAlign': 'center',
                                            'backgroundColor':header_color})
                        ])
                    ]),
                    dbc.Row([
                        dbc.Col([
                            html.Label("The ROI filter and Variable Selection Affect Both Charts")
                        ])
                    ]),
                    #create dropdowns and filters for first row
                    dbc.Row([
                        dbc.Col([html.Label('Please choose number of clusters for model:'),
                            dcc.Input(id="n-clusters",
                                      type='number', 
                                      min=2,
                                      max=10, 
                                      value=3)
                        ], width=4),
                        dbc.Col([
                            html.Label('Please chose max ROI to include in the clustering analysis:'),
                            dcc.Input(id="roi-max",
                                      type='number', 
                                      min=df['ROI'].quantile(.1),
                                      max=df['ROI'].max(), 
                                      value=5)
                        ], width=4),
                        dbc.Col([html.Label('Please choose the other cluster variable for model:'),
                            dcc.Dropdown(id="cluster-variable",
                            options=[{'label': i, 'value': i} for i in numeric_columns if i != 'ROI'],
                            value='Worldwide Gross Income')
                        ], width=4),
                    ]),
                    html.Br(),
                    #Create graphs as second row
                    dbc.Row([
                        dbc.Col([
                            dcc.Graph(id = 'cluster-graph') 
                        ], width=6),
                        dbc.Col([
                            dcc.Graph(id = 'optimal-clusters')
                        ], width=6)
                    ])
                ])
             )
        ])
    ])
])        
        #Fifth tab is linear regression analysis.
        #Here you can select your independent variables, which are categorical
#         dcc.Tab(label='Linear Regression Analysis', children = [
            

    
#ALL CALLBACKS AND FUNCTIONS ARE SORTED IN TAB ORDER
#-------------------------------------------------TAB ONE - TITLE TAB----------------------------------------------#
#Callback to hide input after entering url
@app.callback(
   Output(component_id='title-selector', component_property='style'),
   [Input(component_id='dropdown-to-show_or_hide-element', component_property='value')])

#Function to execute callback of hiding input
def show_hide_element(visibility_state):
    if visibility_state == 'on':
        return {'display': 'block'}
    if visibility_state == 'off':
        return {'display': 'none'}

#Callback to allow for entering of an image to update the title
@app.callback(Output("title-image", "figure"),
              Input("title-selector", "value"))

#Create Title Image
def create_title_image(link):
    if not validators.url(link):
        return {}
    else:
        img = io.imread(link)
        fig_title = px.imshow(img)
        fig_title.update_xaxes(showticklabels=False)
        fig_title.update_yaxes(showticklabels=False)
        fig_title.update_yaxes(automargin=True)
        fig_title.update_xaxes(automargin=True)
        return fig_title


#-------------------------------------------------------TAB TWO----------------------------------------------------#
#First callback and function on first tab
#categorical to update the stacked bar chart
@app.callback(Output("categorical-boxplot", "figure"),
              Input("categorical-variable", "value"),
              Input("filter-roi-categorical", "value")) 

#Write function to give the boxplot of a given variable
def categorical_boxplot(variable_cat, cat_roi_filter):
    
    if cat_roi_filter == None:
        boxplot_df = df
    else:
        boxplot_df = df[df['ROI'] <= cat_roi_filter]

    
    cat_boxplot = px.box(boxplot_df,
                           x=variable_cat,
                           y='ROI')

    cat_boxplot.update_layout(
              xaxis_title_text=variable_cat,
              yaxis_title_text='ROI',
              title={
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'}
                     )

    return cat_boxplot

#Call back for bar chart
@app.callback(Output("categorical-barplot", "figure"),
              Input("categorical-variable", "value"),
              Input("filter-roi-categorical", "value"))   

#Write function to give the bar chart distribution of a given categorical variable
def categorical_barplot(variable_cat, cat_roi_filter):
    if cat_roi_filter == None:
        barplot_df = df
    else:
        barplot_df = df[df['ROI'] <= cat_roi_filter]
        
    
    barplot_grouped = barplot_df.groupby(variable_cat).size().reset_index(name='category count')
    
    fig_bar = px.bar(barplot_grouped, x=variable_cat, y='category count',
                           color_discrete_sequence =['black'])


    fig_bar.update_layout(
              xaxis_title_text=variable_cat,
              yaxis_title_text='Count',
              title={
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'}
                     )

    return fig_bar
  
#Call back for pie chart
@app.callback(Output("categorical-piechart", "figure"),
              Input("categorical-variable", "value"),
              Input("filter-roi-categorical", "value"))   


def categorical_piechart(variable_cat, cat_roi_filter):
    if cat_roi_filter == None:
        pieplot_df = df
    else:
        pieplot_df = df[df['ROI'] <= cat_roi_filter]
    
    pieplot_grouped = pieplot_df.groupby(variable_cat).size().reset_index(name='category count')
    
    
    fig_pie = px.pie(pieplot_grouped, values='category count', names=variable_cat)


    fig_pie.update_layout(
              title={
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'}
                     )
    fig_pie.update_traces(textposition='inside')
    fig_pie.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
    
    return fig_pie
                     
                     
#-------------------------------------------------------TAB THREE----------------------------------------------------#
#First app is numeric x axis and filter by roi callbacks on the scatterplot
@app.callback(Output("numeric-histogram", "figure"),
              Input('x-axis-numeric', 'value')) 

#Return histogram by different columns
def histogram_numeric(variable):
    fig_histogram = px.histogram(df, x=variable, nbins= 50)
    
    fig_histogram.update_layout(
            title={
                'text': f"{variable}",
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'})
    
    return fig_histogram

#Second chart is numeric x axis and filter by roi callbacks on the scatterplot
@app.callback(Output("relationship-scatterplot-graph", "figure"),
              Input("max-roi-numerics", "value"), 
              Input('x-axis-numeric', 'value'))   


#Function that filters for the above callback.
def create_scatter_figure(roi, x_axis):
    filtered_df = df[df['ROI'] <= roi]
                               
        
    fig_scatter = px.scatter(filtered_df, x=x_axis,
                            y='ROI',
                            trendline="ols",
                                )

    fig_scatter.update_layout(
        xaxis_title_text=x_axis,
        yaxis_title_text='ROI',
        font=dict(size=12,color='black'),
        plot_bgcolor='white')

    fig_scatter.update_yaxes(ticks='outside', tickcolor='black')
    fig_scatter.update_xaxes(ticks='outside', tickcolor='black')

    fig_scatter.add_vline(x=0, line_color='black', line_width=1)
    fig_scatter.add_hline(y=0, line_color='black', line_width=1)

    fig_scatter.update_layout(
        title={
            'text': f"Effect of {x_axis} on ROI",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
        
    
    return fig_scatter

#------------------------------------------------------------TAB FOUR------------------------------------------------#

#Callback for determining number of clusters and highest roi value to include in cluster analysis
@app.callback(
    Output("cluster-graph", "figure"),
    Input('roi-max', 'value'),
    Input('n-clusters', 'value'),
    Input('cluster-variable', 'value'))    

#Function to run cluster analysis and graph results using budget and roi as the variables
def cluster_analysis(roi_value, number_clusters, cluster_variable):
    #Create cluster dataframe
    cluster_df = df.copy()
    
    if roi_value == None:
        cluster_df = cluster_df
    else:
        cluster_df = cluster_df.loc[cluster_df['ROI'] < roi_value]
    
    if cluster_df[cluster_variable].isnull().sum() > 0:
        cluster_df = cluster_df.loc[cluster_df[cluster_variable].notnull()]
    else:
        cluster_df = cluster_df
    
    #Standardize columns for clustering
    for i in ['ROI', cluster_variable]:
        cluster_df[i+'_z'] = (cluster_df[i] - cluster_df[i].mean())/cluster_df[i].std() 
    
    if number_clusters == None:
        number_clusters = 2
    else:
        number_clusters = number_clusters
    
    #run cluster analysis with n clusters
    model_clustered = cluster.KMeans(n_clusters = number_clusters, random_state = 10)
    fitted_models_cluster = model_clustered.fit(cluster_df[['ROI_z',cluster_variable+'_z']])
    fitted_models_cluster.labels_
    
    #add cluster to original dataframe
    cluster_df['cluster'] = fitted_models_cluster.labels_.astype(str)
    fig_cluster_roi = px.scatter(cluster_df, x=cluster_variable,
                                y='ROI',
                                color='cluster')

    fig_cluster_roi.update_layout(
            xaxis_title_text=cluster_variable,
            yaxis_title_text='ROI',
            font=dict(size=12,color='black'),
            plot_bgcolor='white')

    fig_cluster_roi.update_yaxes(ticks='outside', tickcolor='black')
    fig_cluster_roi.update_xaxes(ticks='outside', tickcolor='black')

    fig_cluster_roi.add_vline(x=0, line_color='black', line_width=1)
    fig_cluster_roi.add_hline(y=0, line_color='black', line_width=1)

    fig_cluster_roi.update_layout(
        title={
            'text': f"{number_clusters} Clusters of Movies by ROI and {cluster_variable}",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    
    return fig_cluster_roi
                                             
                                             
#Second chart is the elbow graph to show optimal clusters
#Call back matches analysis above roi with determining the optimal clusters
@app.callback(Output("optimal-clusters", "figure"),
              Input("roi-max", "value"),
              Input('cluster-variable', 'value'))

#Graph of optimal clusters
def optimal_cluster_graph(roi_value, cluster_variable):
    cluster_df_optimal = df.copy()
    
    if cluster_df_optimal[cluster_variable].isnull().sum() > 0:
        cluster_df_optimal = cluster_df_optimal.loc[cluster_df_optimal[cluster_variable].notnull()]
    else:
        cluster_df_optimal = cluster_df_optimal
    
    if roi_value == None:
        cluster_df_optimal = cluster_df_optimal
    else:
        cluster_df_optimal = cluster_df_optimal.loc[cluster_df_optimal['ROI'] < roi_value]
    
    #Standardize columns for clustering
    for i in ['ROI', cluster_variable]:
        cluster_df_optimal[i+'_z'] = (cluster_df_optimal[i] - cluster_df_optimal[i].mean())/cluster_df_optimal[i].std()
    
    #Create dataframe to measure optimal number of clusters
    inertia_list = []

    for i in range(2,11):
        model = cluster.KMeans(n_clusters = i, random_state = 10)
        fitted_models = model.fit(cluster_df_optimal[['ROI_z',cluster_variable+'_z']])
        inertia_list.append(fitted_models.inertia_)                                         
    
                                             
    #Visualize elbow method of ideal number of clusters
    elbow_df = pd.DataFrame({'SS':inertia_list,
                      'Number of Clusters':range(2,11)})
    
                                             
    fig_optimal_clusters = px.line(elbow_df, x="Number of Clusters", y="SS")
                                             
    fig_optimal_clusters.update_layout(
        title={
            'text': f"Optimal Number of Clusters",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    
    return fig_optimal_clusters                                         

    
    
    
# def model_summary(x_variables, categorical_dictionary):    
#     drop_columns = [key + '_' + value for key, value in variable_drop_cat_dict.items()]
#     x = df.copy()
#     x = x[x_columns]
#     x = pd.get_dummies(x, columns=variable_drop_cat_dict.keys())
    
#     x = x.drop(drop_columns, axis=1)
#     x = sm.add_constant(x)
    
#     y = df.copy()
#     y = y[[y_column]]
    
    
#     model = sm.OLS(y,x)
#     results = model.fit()
#     results_summary = results.summary()

#     #Note that tables is a list. The table at index 1 is the "core" table.
#     Additionally, read_html puts dfs in a list, so we want index 0
#     results_as_html = results_summary.tables[1].as_html()   
#     return results_as_html

app.run_server(debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
